<a href="https://www.kaggle.com/code/mikedelong/animal-welfare-data-eda?scriptVersionId=144452081" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
from glob import glob
files = sorted(list(glob(pathname='/kaggle/input/animal-welfare/*.csv')))
files

In [ ]:
import pandas as pd
from plotly.express import bar

for filename in files[:3]:
    df = pd.read_csv(filepath_or_buffer=filename, thousands=',').drop(columns=['Code', 'Year'])
    bar(data_frame=df,x=df.columns[0], y=df.columns[1], log_y=True,).show()

We use a log scale for the y axis because the quantities are of such different scale.

In [ ]:
laying_df = pd.read_csv(filepath_or_buffer=files[3], thousands=',')
laying_df.head()

In [ ]:
from plotly.express import scatter
scatter(data_frame=laying_df, x='Number of hens in cages', y='Number of cage-free hens', color='Year', hover_name='Entity', log_x=True)